In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
import math

In [ ]:
# 유튜브 구독정보 난수생성기로 임의 생성

class RandomGenerator(object):
   
    def __init__(self, random_state=1):
        """
        -매개변수-
        random_state : 가중치 초기화를 위한 무작위 난수 생성기의 seed
        """
        self.random_state = random_state
        
    def GeneratingRandomData(self, loc=4, size=5000, scale=3, n_features=10):
        """
        난수생성기로 임의 데이터 생성
        """
    
        np.random.seed(self.random_state)
        # 특정 seed를 가지는 np.random.RandomState() 객체를 만들고, 이 객체를 통해서 난수 생성
        self.data = np.random.normal(loc, scale, size)
        for i in range(len(self.data)):
            self.data[i] = (abs(int(self.data[i])))
                
        return self.data.reshape(-1, n_features)
    
    def GeneratingFeature(self): # feature 임의 생성
        self.feature = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]
    
    def ArrayToDataframe(self, random_array, n_sample):
        return pd.DataFrame(random_array, index=range(0, n_sample), columns=self.feature)

In [ ]:
# 난수생성기 객체 생성

dataGenerator = RandomGenerator(156)

In [ ]:
randomData = dataGenerator.GeneratingRandomData()
randomData.shape

In [ ]:
# feature 이름 리스트 구성

feature_names = ['Game', 'Cook', "Travel", "Money", "Academy", "Learning", "Sing", "Sports", "Machine", "Mistery"]

In [ ]:
dataGenerator.GeneratingFeature()

In [ ]:
dfRandomData = dataGenerator.ArrayToDataframe(randomData, 500)
dfRandomData

In [ ]:
class Classification(object):
    
    def __init__(self, df_data):
        self.df = df_data
    
    def Extraction(self):
        # 가장 선호하는 feature 추출
        self.ls = []
        for i in range(0, self.df.shape[0]):
            self.ls.append(self.df.sort_values(by=[i], axis=1).iloc[i, -2:].index.tolist())
            
        # 각 이용자의 구독 채널 수 추출
        arr = np.array(self.df)
        self.num_ch = np.sum(arr, axis=1).reshape(-1, 1)
        
    def Ohe(self): # 각 샘플에 대해 추출한 가장 선호하는 2개의 특성에 대해 원핫인코딩
        ohe = OneHotEncoder(sparse=False)
        return ohe.fit_transform(self.ls)
    
    def makeData(self, train_feature):
        self.X_train = np.hstack((self.num_ch, train_feature))
        
    def Kmeans(self): # k_means 클러스터링
        model = KMeans(n_clusters=16)
        model.fit(self.X_train)
        return np.array(model.predict(self.X_train))
    
    def euclidean_distance(pt1, pt2): # 데이터 포인트 거리 측정
      distance = 0
      for i in range(len(pt1)):
        distance += (pt1[i] - pt2[i]) ** 2
      return distance ** 0.5
    
    def ReturnResult(self, k=5): 
        # 각 이용자마다 가장 가까운 k개의 이용자 매칭 결과 반환
        # 현재 시간복잡도가 O(n^2)이므로 계산량이 많음 -> 추후 시간복잡도 감소 필요
        
        #k = 5
        k_users = np.empty((0,k))
        for i in range(0, self.X_train.shape[0]):
            distance = []
            for j in range(0, self.X_train.shape[0]):
                if i != j:
                    distance.append(euclidean_distance(self.X_train[i, :], self.X_train[j, :]))
                if j == (self.X_train.shape[0] - 1):
                    # i 번째 데이터 포인터와 다른 데이터들 사이의 거리를 작은순으로 정렬한 후 해당 인덱스를 이용해서 가장 가까운 k명의 이용자 선별
                    distance_idx = np.array(distance).argsort()
                    k_users = np.append(k_users, [distance_idx[:k]], axis=0)
                    
        return k_users

In [ ]:
cl = Classification(dfRandomData)

In [ ]:
cl.Extraction()
cl.ls

In [ ]:
train_feature = cl.Ohe()
train_feature

In [ ]:
cl.makeData(train_feature)

In [ ]:
result = cl.Kmeans()
result

In [464]:
k_users = cl.ReturnResult()

In [465]:
k_users

array([[126., 121., 116., 222.,   2.],
       [379., 181., 170., 118., 440.],
       [370., 441., 434., 230., 215.],
       ...,
       [404., 165., 383., 413., 473.],
       [234., 121., 483.,  98., 230.],
       [164.,  87., 287., 131., 146.]])